In [ ]:
%matplotlib inline

import xml.etree.cElementTree as etree
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

file1 = "data/parametersweepNGM.xml"

#### blueish colors
cdict = {'red': ((0.,0,0),(0.33,0,0),(0.66,0,0),(1.0,1,1)), 
    'green': ((0.,0,0),(0.33,0,0),(0.66,1,1),(1.0,1,1)),
    'blue': ((0.,0,0),(0.33,1,1),(0.66,1,1),(1.,1,1))}

ckeys = cdict.keys()
alpha = 1.0
cdict_alpha = {}
def scale_ctuple((a,b,c),x):
    return (a,1-(1-b)*x,1-(1-c)*x)

for ckey in ckeys:
    cdict_alpha[ckey] = map(lambda x: scale_ctuple(x,alpha),cdict[ckey])

my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict_alpha,256)

def fst(p): return (lambda (x,y) : x)(p)
def snd(p): return (lambda (x,y) : y)(p)

def valueMean(vals):
    vals = [snd(val) for val in vals]
    return np.mean(vals)

def transpose(M):
    return [[row[i] for row in M] for i in range(0,len(M[0]))]

def MakeParSweepPlot(ax,fileName):
    tree = etree.parse(fileName)
    root = tree.getroot()
    stableDistSamples = root.findall("stable_dist_sample")
    stableDistSamples = [x.attrib for x in stableDistSamples]
    params2 = stableDistSamples[0]
    lambda_acute = float(params2["LAMBDA1"])
    lambdas = [float(x["PESC"]) * lambda_acute for x in stableDistSamples]
    loglambdas = [np.log(x)/np.log(10) for x in lambdas]
    vlmaxs = [float(x["VLMAX"]) for x in stableDistSamples]
    spVLtavs = [float(x["spVLtavM1"]) for x in stableDistSamples]
    xs = lambdas
    ys = vlmaxs
    zs = spVLtavs
    xis = sorted(list(set(xs)))
    yis = sorted(list(set(ys)))
    zipped = zip(zip(xs,ys),zs)
    fullGrid = [[filter(lambda ((x,y),z): y == yi and x == xi, zipped) for yi in yis] for xi in xis]
    zis = [[valueMean(x) for x in row] for row in fullGrid]
    zis = transpose(zis)
    stepsize = 0.5
    maxz = max(map(max,zis))
    minz = min(map(min,zis))
    myLevels = np.arange(-1.5,11.0,stepsize)
    xis, yis = np.meshgrid(xis, yis)
    
    CSreturn = ax.contourf(xis,yis,zis,levels=myLevels,cmap=my_cmap)
    txtcols = ['white' if ell < 4.5 else 'black' for ell in myLevels]
    CS = ax.contour(xis,yis,zis,levels=myLevels,colors=txtcols)
    zc = CS.collections[12]
    plt.setp(zc, linewidth=3)
    ax.set_xscale("log")
    ax.clabel(CS, fontsize=int(9), inline=1, fmt='%1.1f')
    ax.set_xlabel("mutation rate ($\\lambda_{1,{\\rm esc}}$)")
    ax.set_ylabel("maximal virus load ($V_{\\rm max}$)")
    ax.set_yticks([4,5,6,7,8,9,10])
    return CSreturn



figscale = 1.0
fig = plt.figure(figsize=[8,7])
gs = GridSpec(1, 1,width_ratios=[1])
ax1 = fig.add_subplot(gs[:, :-1])

CS = MakeParSweepPlot(ax1,file1)

cbar = fig.colorbar(CS)
cbar.set_label('$\\langle {\\rm spVL}\\rangle$', rotation=90)

fig.savefig("parametersweep-plot.png")